# Ottimizzazione dei parametri

In [27]:
import pandas as pd
from scipy.integrate import odeint
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import random
from statistics import mean 
import itertools
import ga 
import math
path_file="https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv"
dataset=pd.read_csv(path_file,parse_dates=['data'])

# guariti,infetti,sani, morti 
dataset['asintomatici']= dataset['totale_casi'] * 0.44 
dataset['immuni'] = dataset['dimessi_guariti'] * 0.05  #verificare proporzione
data=dataset[['totale_positivi','dimessi_guariti','deceduti','asintomatici','immuni']]


def initializeParameters(minimo,massimo,n=9):
        return tuple([random.uniform(minimo,massimo) for i in range(0,n)])

def deriv(y,t,k1,k2,k3,k4,k5,k6,k7,k8,k9):  
    '''set di equazioni differenziali'''
    Sani,Infetti,Asintomatici,Morti,Guariti,Immuni = y 
    dSanidt =  -k1*Sani*Infetti -k2*Sani*Infetti-k3*Sani*Asintomatici- k4*Sani*Asintomatici
    dInfettidt = k1*Sani*Infetti+k4*Sani-k6*Infetti-k7*Infetti
    dAsintomaticidt =k2*Sani*Infetti+k3*Sani*Asintomatici-k5*Asintomatici
    dMortidt=k6*Infetti
    dGuaritidt = k5*Asintomatici+k7*Infetti-k8*Guariti-k9*Guariti
    dImmunidt = k9*Guariti
    return dSanidt,dInfettidt,dAsintomaticidt,dMortidt,dGuaritidt,dImmunidt  

def perturbazione(ls):
    sup = 0.2
    for i in range(len(ls)):
        if ls[i] < 0.5:
            ls[i]=random.uniform(ls[i],ls[i]+sup)
        else:
            ls[i]=random.uniform(0,ls[i]-sup)
    return(ls)

def diffusione(parameters,Sani = 6036000, Infetti=221,Guariti=1, Morti= 7,Immuni = 8,Asintomatici=10,orizzonte_temporale_in_gg=data.shape[0]):
        y0 =  [Sani,Infetti,Morti,Guariti,Immuni,Asintomatici]
        t = np.linspace(0,orizzonte_temporale_in_gg,orizzonte_temporale_in_gg)
        ret = odeint(deriv, y0, t,args=parameters)
        return(ret)

def fitness(previsti):
        df=pd.DataFrame(previsti,columns=['sani','totale_positivi','dimessi_guariti','deceduti','asintomatici','immuni'])
        df=df.drop(columns=['sani'])
        dataframe=((df-data)**2)
        lista=dataframe.values.tolist()
        merged = list(itertools.chain.from_iterable(lista))
        media=mean(merged)
        return media 

def localsearch(iniziale,iterazioni,minimo,massimo,param_only=True):
        #list(initializeParameters(minimo,massimo))
        k=iniziale.copy()
        errore= math.inf
        val='1'
        set_ottimo=[]
        for c in range(iterazioni):
            valori=diffusione(tuple(k))
            stimaerrore=fitness(valori)
            if stimaerrore<errore:
                errore=stimaerrore
                val=valori 
                set_ottimo=iniziale
            iniziale = list(initializeParameters(minimo,massimo))
            k=iniziale.copy()
        if param_only==True : 
            return set_ottimo
        else : 
            return (set_ottimo,val,errore)

def ils(iterazioni=10,minimo=0.00001,massimo=0.001,param_only=True):
        iniziale = list(initializeParameters(minimo,massimo))
        loc_search = localsearch(iniziale,iterazioni,minimo,massimo,param_only=False)
        k=loc_search[0]
        best_val = loc_search[1]
        best_errore = loc_search[2]
        val=0
        for c in range(iterazioni):
            val=best_val
            nuova_iniziale = perturbazione(k)
            loc_search = localsearch(nuova_iniziale,iterazioni,minimo,massimo,param_only=False)
            val = loc_search[1]
            if loc_search[2] < best_errore: # Se nuovo errore minore best si aggiorna il best
                best_errore = loc_search[2]
                best_val= loc_search[1]
                k=loc_search[0]
        if param_only == True : 
            return k
        else : return k,best_val,best_errore  




In [49]:
ils()

[0.28631806517741815,
 0.08379106292299272,
 0.35632564539013395,
 0.4928745441424222,
 0.16700924299800604,
 0.18589290088107546,
 0.4526659972145615,
 0.18295030029963563,
 0.38857911167472464]